# Shortest Paths

간선에 가중치 있는, 방향이 있는 그래프

In [ ]:
V = {"A", "B", "C", "D", "E", "F", "G", "H"}
E = { "A" : {"B":8, "C":11, "D":9},
      "B" : {"E":10},
      "C" : {"F":8, "G":8},
      "D" : {"B":6, "C":3, "E":1},
      "E" : {"H":2},
      "F" : {"G":7},
      "G" : {"D":12, "H":5},
      "H" : {"F":4}
      }

## Dijkstra
- 시작점으로부터, 가장 가까운 노드를 합쳐나간다
- 이 때, 새롭게 합쳐지는 노드 주변 노드들의 거리를 업데이트한다

In [25]:
import sys

def Dijkstra(node):
  S = set()
  d = {}
  for v in V:
    d[v] = sys.maxsize
  d[node] = 0

  while S!=set(V):
    new = extract_min(V-S, d) # S에 포함되지 않고, 가장 가까운 노드를
    S.add(new) # S에 추가한다

    for adj_node in E[new]: # 추가된 노드의 근처 노드에 대해
      if adj_node in V-S and \
        d[adj_node] > d[new] + E[new][adj_node]: 
        # S에 포함되어있지 않고, 
        # 거리가, new 거처가는 거리보다 크면
        d[adj_node] = d[new] + E[new][adj_node]
        # new 거쳐가는 거리를 할당한다(업데이트 한다, 이완한다))
        print(new, '-', adj_node)
  
  return d

def extract_min(Q, d):
  nearest_node = None
  for node in Q:
    if nearest_node is None or d[node] < d[nearest_node]:
      nearest_node = node
  return nearest_node


Dijkstra('A')


A - B
A - C
A - D
B - E
D - E
E - H
C - F
C - G
H - F


{'B': 8, 'H': 12, 'F': 16, 'C': 11, 'E': 10, 'A': 0, 'D': 9, 'G': 19}

## Bellman-Ford
- 한 단계가 오를 때 마다, 근처(한 칸)의 노드들을 스캔한다.
- 해당 노드들까지의 거리를 단계마다 업데이트한다.

- 음의 가중치를 허용한다. 단, 음의 싸이클은 없어야 한다.

In [ ]:
V2 = {"A", "B", "C", "D", "E", "F", "G", "H"}
E2 = { "A" : {"B":8, "C":11, "D":9},
      "B" : {"E":10},
      "C" : {"F":8, "G":8},
      "D" : {"B":-15, "C":3, "E":1},
      "E" : {"H":2},
      "F" : {"G":-7},
      "G" : {"D":12, "H":5},
      "H" : {"F":4}
      }

In [67]:
import sys

def bellman_ford(node):
  d = {}
  for v in V2:
    d[v] = sys.maxsize
  d[node] = 0

  for i in range(0, len(V)-1): # 단계별로 1칸 씩 확장한다
    for start in E: 
      for end in E[start]: # start: 확장 전 노드, end: 확장 후 노드
        if d[end] > d[start] + E2[start][end]:
          d[end] = d[start] + E2[start][end]
    print(i, d)
  
  return dict(sorted(d.items()))

bellman_ford('A')    


0 {'B': -6, 'H': 12, 'F': 16, 'C': 11, 'E': 10, 'A': 0, 'D': 9, 'G': 12}
1 {'B': -6, 'H': 6, 'F': 10, 'C': 11, 'E': 4, 'A': 0, 'D': 9, 'G': 9}
2 {'B': -6, 'H': 6, 'F': 10, 'C': 11, 'E': 4, 'A': 0, 'D': 9, 'G': 3}
3 {'B': -6, 'H': 6, 'F': 10, 'C': 11, 'E': 4, 'A': 0, 'D': 9, 'G': 3}
4 {'B': -6, 'H': 6, 'F': 10, 'C': 11, 'E': 4, 'A': 0, 'D': 9, 'G': 3}
5 {'B': -6, 'H': 6, 'F': 10, 'C': 11, 'E': 4, 'A': 0, 'D': 9, 'G': 3}
6 {'B': -6, 'H': 6, 'F': 10, 'C': 11, 'E': 4, 'A': 0, 'D': 9, 'G': 3}


{'A': 0, 'B': -6, 'C': 11, 'D': 9, 'E': 4, 'F': 10, 'G': 3, 'H': 6}

In [ ]:
# TODO: DP로 구현하기

## Floyd-Warshall
- 모든 노드에 대한 최소 경로 찾기

In [129]:
import math
def fw():
  d = init_d() # from-to 테이블 초기화
  for mid in range(0, len(V2)): # 중간경로 거친 것이 더 가까운지 계산하는 로직
    for start in range(0, len(V2)):
      for end in range(0, len(V2)):
        d[start][end] = min(d[start][end], d[start][mid] + d[mid][end])
    # print_arr(d)
  return d

def init_d(): 
  d = []
  for start in sorted(V2):
    line = []
    for end in sorted(V2):
      if start is end:
        line.append(0) # 같은 노드는 0
      elif end in E2[start]:
        line.append(E2[start][end]) # 가중치 있으면 업데이트
      else:
        line.append(math.inf) # 가중치 없으면 inf
    d.append(line)
  return d

def print_arr(arr):
  print('='*36)
  for idx, y in enumerate(arr):
    if idx==0: # to
      print("f\\to", end=" ")
      for v in sorted(V2):
        print(f'%3c' %v, end=" ")
      print()
    print(f'%3c' %sorted(V2)[idx], end="  ") #from
    for x in y:
      print(f'%3.0f' %x, end=" ")
    print()
  print('='*36)

print_arr(fw())


f\to   A   B   C   D   E   F   G   H 
  A    0  -6  11   9   4  10   3   6 
  B  inf   0  24  21  10  16   9  12 
  C  inf  -2   0  13   8   8   1   6 
  D  inf -15   3   0  -5   1  -6  -3 
  E  inf  -4  14  11   0   6  -1   2 
  F  inf -10   8   5   0   0  -7  -2 
  G  inf  -3  15  12   7   9   0   5 
  H  inf  -6  12   9   4   4  -3   0 


##  DAG

In [ ]:
# TODO

## SCC

In [ ]:
# TODO